In [22]:
import time
import mysql.connector
import pandas as pd
from tabulate import tabulate  # Import tabulate for a better table format

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Aliza.123",  # Replace with your actual MySQL password
    database="bookstore_db"
)
cursor = conn.cursor()

# Ensure tables exist
cursor.execute("""
    CREATE TABLE IF NOT EXISTS authors (
        AuthorID INT AUTO_INCREMENT PRIMARY KEY,
        AuthorName VARCHAR(255) UNIQUE
    )
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS books (
        BookID INT AUTO_INCREMENT PRIMARY KEY,
        Title VARCHAR(255) UNIQUE
    )
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS bookstore (
        StoreID INT AUTO_INCREMENT PRIMARY KEY,
        BookID INT,
        AuthorID INT,
        Price DECIMAL(10,2),
        Rating FLOAT,
        FOREIGN KEY (BookID) REFERENCES books(BookID),
        FOREIGN KEY (AuthorID) REFERENCES authors(AuthorID)
    )
""")
conn.commit()

print("✅ Database and tables are ready!")

# Load book data from CSV
df = pd.read_csv("amazon_data_engineering_books.csv")

# Insert Data into MySQL Tables
for index, row in df.iterrows():
    if pd.notnull(row['Title']) and pd.notnull(row['Author']) and pd.notnull(row['Rating']) and pd.notnull(row['Price']):
        # Insert Author
        cursor.execute("INSERT IGNORE INTO authors (AuthorName) VALUES (%s)", (row['Author'],))
        conn.commit()

        # Retrieve AuthorID
        cursor.execute("SELECT AuthorID FROM authors WHERE AuthorName = %s", (row['Author'],))
        author_id = cursor.fetchone()[0]

        # Insert Book
        cursor.execute("INSERT IGNORE INTO books (Title) VALUES (%s)", (row['Title'],))
        conn.commit()

        # Retrieve BookID
        cursor.execute("SELECT BookID FROM books WHERE Title = %s", (row['Title'],))
        book_id = cursor.fetchone()[0]

        # Insert into bookstore (Avoid duplicates)
        cursor.execute("""
            INSERT IGNORE INTO bookstore (BookID, AuthorID, Price, Rating)
            VALUES (%s, %s, %s, %s)
        """, (book_id, author_id, row['Price'], row['Rating']))
        conn.commit()

print("✅ Data successfully inserted into MySQL!")

# SQL query to fetch books published after 2015 and price below $50
sql_query = """
    SELECT authors.AuthorName, books.Title, bookstore.Rating, bookstore.Price
    FROM bookstore
    JOIN books ON bookstore.BookID = books.BookID
    JOIN authors ON bookstore.AuthorID = authors.AuthorID
    WHERE bookstore.Price < 50
    ORDER BY bookstore.Price ASC
    LIMIT 15;
"""

# Measure MySQL query execution time
start_time = time.time()
cursor.execute(sql_query)
sql_results = cursor.fetchall()
sql_time = time.time() - start_time  # ✅ Measures query speed

# Convert results to DataFrame
df_sql = pd.DataFrame(sql_results, columns=["Author", "Title", "Rating", "Price"])

# Display results
print("\n📌 Filtered Books from MySQL:\n")
print(tabulate(df_sql, headers="keys", tablefmt="psql"))  # Formats output as a clean table

# Print execution time
print(f"📌 MySQL Query Execution Time: {sql_time:.5f} seconds")

# Close connection
cursor.close()
conn.close()


✅ Database and tables are ready!
✅ Data successfully inserted into MySQL!

📌 Filtered Books from MySQL:

+----+----------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+---------+
|    | Author                                                                           | Title                                                                                                                                                                              |   Rating |   Price |
|----+----------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+---------|
|  0 | Book 3 of 3: